<a href="https://colab.research.google.com/github/Jack3690/pydol/blob/main/notebooks/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/Jack3690/pydol/ -q
!pip install astroquery -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.2.1 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.1 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.
langchain 0.3.14 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.2.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.1 which is incompatible.
pytensor 2.26.4 requires numpy<2,>=1.17.0, but you have numpy 2.2.1 which is incompatible.
tensorflow 2.17.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.2.1 which is incompatible.
thinc 8.2.5 requires numpy<2.0.0,>=1.19.0; python_version >= "3.9", but you have numpy 2.2.1 which is incompatible.


In [4]:
!pip install  numpy==1.25.2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 78.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
drizzlepac 3.9.0 requires numpy>2.0, but you have numpy 1.25.2 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.
stwcs 1.7.4 requires numpy>=2.0, but you have numpy 1.25.2 which is incompatible.


In [2]:
!pip install asdf==2.15.2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.8/961.8 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gwcs 0.22.1 requires asdf>=3.3.0, but you have asdf 2.15.2 which is incompatible.


# **DOLPHOT Installation**

In [ ]:
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.tar.gz
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.NIRCAM.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F140M.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F212N.tar.gz

In [ ]:
!tar -xf dolphot2.0.tar.gz
!tar -xf dolphot2.0.NIRCAM.tar.gz
!tar -xf nircam_F140M.tar.gz
!tar -xf nircam_F212N.tar.gz

In [ ]:
with open('/content/dolphot2.0/Makefile') as f:
  dat = f.readlines()
dat[28]= dat[28][1:]

with open('/content/dolphot2.0/Makefile','w') as f:
  f.writelines(dat)

In [ ]:
cd /content/dolphot2.0/

In [ ]:
import os

In [ ]:
os.system('make all')

In [ ]:
cd /content/

In [ ]:
if 'dolphot2.0' not in os.environ['PATH']:
  os.environ['PATH'] += ':/content/dolphot2.0/bin'
  os.environ['PATH'] += ':/content/dolphot2.0/nircam'

# **Imports**

In [6]:
from glob import glob
import astropy.io.fits as fits

import os
import astroquery
from astroquery.mast import Observations
import pandas as pd

from astropy.table import Table, vstack
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import json
from astropy.coordinates import angular_separation
import astropy.units as u
#from pydol.pipeline.jwst import jpipe
import multiprocessing as mp

# **Data**

**Observation ID format**

jw\<ppppp>\<ooo><vvv>\_\<gg>\<s>\<aa>\_\<eeeee>(-<"seg"NNN>)\_\<detector>\_\<prodType>.fits

where

ppppp: program ID number

ooo: observation number

vvv: visit number

gg: visit group

s: parallel sequence ID (1=prime, 2-5=parallel)

aa: activity number (base 36)

eeeee: exposure number

segNNN: the text “seg” followed by a three-digit segment number (optional)

detector: detector name (e.g. ‘nrca1’, ‘nrcblong’, ‘mirimage’)

prodType: product type identifier (e.g. ‘uncal’, ‘rate’, ‘cal’)

In [7]:

#Example data for F200W filter for NGC628
obs_table = Observations.query_criteria(proposal_id=["1783"],objectname="NGC628",filters= ['F115W', 'F150W', 'F200W'])[3:]

t = [Observations.get_product_list(obs) for obs in obs_table]
files = unique(vstack(t), keys='productFilename')

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/data/stage0/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

manifest = Observations.download_products(
           files,
           productSubGroupDescription='UNCAL',
           filters=['F200W'],
           calib_level=[1],
           download_dir = data_dir,
           flat = True,
           )

NameError: name 'unique' is not defined

In [ ]:
#Example data for NGC628

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/photometry/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

manifest = Observations.download_products(
           files,
           productSubGroupDescription='I2D',
           filters=['F200W'],
           calib_level=[3],
           download_dir = data_dir,
           flat = True,
           )


# **Pipeline**

In [ ]:
# For reducing load on the RAM. Let's split pipeline into two steps

input_files = glob(f'/content/data/stage0/*b1*')
out_dir = '/content/'
jwst_data = jpipe(input_files, out_dir)
jwst_data()

# **DOLPHOT Photometry**

In [ ]:
if 'dolphot2.0' not in os.environ['PATH']:
  os.environ['PATH'] += ':/content/dolphot2.0/bin'
  os.environ['PATH'] += ':/content/dolphot2.0/nircam'

In [ ]:
crf_files = glob(f'/content/data/stage3/*b1*crf.fits')

In [ ]:
from pydol.photometry import nircam

In [ ]:
nircam.nircam_phot(crf_files,
                   filter='f212n',
                   output_dir='/content/photometry/',
                   drz_path='/content/photometry/jw01701-o052_t007_nircam_clear-f212n-sub640_i2d',
                   cat_name= 'b1')